In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from UTILS import time_series_henColumn_tsRow, ts_compare_session, TimeSeriesPlot
import config_mobility as config

we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [3]:
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
nbr_sec = config.nbr_sec

## Download data

In [4]:
df_master = pd.read_csv(os.path.join(path_initial_data,'Mo_Masterfile.csv'), sep=';') #, parse_dates=['date']) wrong!
df_master.rename(columns={'time':'Timestamp','day':'DayID','hen':'HenID','date':'day'}, inplace=True)
#dico or join by hen (not day as some are nan but we actually know then)
df_master['day'] = df_master['day'].map(lambda x: dt.datetime.strptime(x, "%d.%m.%Y"))
df_master['HenID'] = df_master['HenID'].map(str)
dico_hen_group = dict(zip(df_master['HenID'].tolist(), df_master['group'].tolist()))
#df_master.filter(['HenID','group'])
print(df_master.shape)
df_master.head(3)

(7920, 43)


,HenID,timepoint,group,age,DayID,day,file,severity,pen,hybrid,...,meanchange,mean1,medianchange,transitions,real0,sumdur,nest_vs_total,severity_diff,KBF_new,indicator
0,1,1,A,21,1,2016-10-27,1A01,0.0,1,LSL,...,1.23,1.0,1.0,91.0,NaN,57067.0,0.177419,NaN,0.0,NaN
1,1,1,A,21,2,2016-10-28,1A01,0.0,1,LSL,...,1.28,1.0,1.0,108.0,NaN,56854.0,0.171053,0.0,0.0,NaN
2,1,1,A,21,3,2016-10-29,1A01,0.0,1,LSL,...,1.17,1.0,1.0,128.0,NaN,56721.0,0.255319,0.0,0.0,NaN


In [5]:
df_master.groupby(['group'])['hybrid'].agg(lambda x: Counter(x)).reset_index()

,group,hybrid
0,A,"{'LSL': 2640, 'LB': 1320}"
1,B,"{'LSL': 1320, 'LB': 2640}"


In [23]:
df = pd.read_csv(os.path.join(path_initial_data,'movements_timeseries.csv'), sep=';') #, parse_dates=['time'])
df.rename(columns={'time':'Timestamp','chicken':'HenID','current_Level':'zone'}, inplace=True)
df['Timestamp'] = df['Timestamp'].map(lambda x: dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df['day'] = df['Timestamp'].map(lambda x: dt.datetime(x.year, x.month, x.day))
df['HenID'] = df['HenID'].map(str)
df['zone'] = df['zone'].map(str)
df['group'] = df['HenID'].map(dico_hen_group)
df['SessionID'] = df['sessionNumber'].map(str)+df['group']
df['zone'] = df['zone'].map(lambda x: 'zone_'+x)
print(df.shape)
df.head(3)

(362612, 7)


,Timestamp,zone,HenID,sessionNumber,day,group,SessionID
0,2016-10-27 01:30:00,zone_4,1,1,2016-10-27,A,1A
1,2016-10-27 04:29:17,zone_5,1,1,2016-10-27,A,1A
2,2016-10-27 04:38:59,zone_4,1,1,2016-10-27,A,1A


## Save Movement record csv

In [24]:
#merge to get the severity link to sessionNumber
print(df.shape, df_master.shape)
df = pd.merge(df, df_master.filter(['HenID','day','severity','hybrid']), how='left', on=['HenID','day'])
df['HenID'] = df['HenID'].map(lambda x: 'hen_'+x)
print(df.shape)
df.head(3)

(362612, 7) (7920, 43)
(362612, 9)


,Timestamp,zone,HenID,sessionNumber,day,group,SessionID,severity,hybrid
0,2016-10-27 01:30:00,zone_4,hen_1,1,2016-10-27,A,1A,0.0,LSL
1,2016-10-27 04:29:17,zone_5,hen_1,1,2016-10-27,A,1A,0.0,LSL
2,2016-10-27 04:38:59,zone_4,hen_1,1,2016-10-27,A,1A,0.0,LSL


In [25]:
#show rows with at least one nan
df[df.isna().any(axis=1)].shape

(5086, 9)

In [27]:
df.to_csv(os.path.join(path_extracted_data, 'df_movement.csv'), sep=';', index=False)

## Extract time series

In [5]:
df = pd.read_csv(os.path.join(path_extracted_data,'df_movement.csv'), sep=';', parse_dates=['Timestamp'])
print(df.shape)
df.head(3)

(362612, 8)


,Timestamp,zone,HenID,sessionNumber,day,group,SessionID,severity
0,2016-10-27 01:30:00,zone_4,hen_1,1,2016-10-27 00:00:00,A,1A,0.0
1,2016-10-27 04:29:17,zone_5,hen_1,1,2016-10-27 00:00:00,A,1A,0.0
2,2016-10-27 04:38:59,zone_4,hen_1,1,2016-10-27 00:00:00,A,1A,0.0


In [10]:
for i,df_ in tqdm.tqdm(df.groupby(['SessionID'])):
    print('-----------------------------', i)
    time_series_henColumn_tsRow(df_, config, col_ts='zone', name_=i, ts_with_all_hen_value=False, save=True, 
                                hen_time_series=True)

  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

----------------------------- 10A
in this time series there is 49 hens
The initial starting date of the time series is: 2017-07-06 01:30:00, and the ending date will be: 2017-07-11 17:08:29
and after ending the last day at midnight : 2017-07-06 01:30:00, and the ending date will be: 2017-07-11 23:59:59
[0]


  5%|███▋                                                                              | 1/22 [01:43<36:13, 103.52s/it]

----------------------------- 10B
in this time series there is 49 hens
The initial starting date of the time series is: 2017-07-14 01:30:00, and the ending date will be: 2017-07-19 17:08:22
and after ending the last day at midnight : 2017-07-14 01:30:00, and the ending date will be: 2017-07-19 23:59:59
[48  0]


  9%|███████▍                                                                          | 2/22 [03:25<34:19, 102.98s/it]

...

 91%|█████████████████████████████████████████████████████████████████████████▋       | 20/22 [37:45<03:51, 115.53s/it]

----------------------------- 9A
in this time series there is 50 hens
The initial starting date of the time series is: 2017-06-01 01:30:00, and the ending date will be: 2017-06-06 17:06:40
and after ending the last day at midnight : 2017-06-01 01:30:00, and the ending date will be: 2017-06-06 23:59:59
[1 0]


 95%|█████████████████████████████████████████████████████████████████████████████▎   | 21/22 [39:30<01:52, 112.46s/it]

----------------------------- 9B
in this time series there is 50 hens
The initial starting date of the time series is: 2017-06-09 01:30:00, and the ending date will be: 2017-06-14 17:28:12
and after ending the last day at midnight : 2017-06-09 01:30:00, and the ending date will be: 2017-06-14 23:59:59
[0]


100%|█████████████████████████████████████████████████████████████████████████████████| 22/22 [41:13<00:00, 109.55s/it]


## Extract event csv and Flag interesting/not-interesting ones

In [6]:
df = pd.read_csv(os.path.join(path_extracted_data,'df_movement.csv'), sep=';', parse_dates=['Timestamp'])
print(df.shape)
df.head(3)

(362612, 8)


,Timestamp,zone,HenID,sessionNumber,day,group,SessionID,severity
0,2016-10-27 01:30:00,zone_4,hen_1,1,2016-10-27 00:00:00,A,1A,0.0
1,2016-10-27 04:29:17,zone_5,hen_1,1,2016-10-27 00:00:00,A,1A,0.0
2,2016-10-27 04:38:59,zone_4,hen_1,1,2016-10-27 00:00:00,A,1A,0.0


In [10]:
#info on days per session
df[df['HenID']=='hen_6'].groupby(['SessionID'])['day'].agg(lambda x: set(x)).reset_index()

,SessionID,day
0,10A,"{2017-07-07 00:00:00, 2017-07-09 00:00:00, 201..."
1,11A,"{2017-08-03 00:00:00, 2017-08-05 00:00:00, 201..."
2,1A,"{2016-11-01 00:00:00, 2016-10-29 00:00:00, 201..."
3,2A,"{2016-11-19 00:00:00, 2016-11-17 00:00:00, 201..."
4,3A,"{2016-12-09 00:00:00, 2016-12-08 00:00:00, 201..."
5,4A,"{2017-01-06 00:00:00, 2017-01-07 00:00:00, 201..."
6,5A,"{2017-02-03 00:00:00, 2017-02-06 00:00:00, 201..."
7,6A,"{2017-03-05 00:00:00, 2017-03-07 00:00:00, 201..."
8,7A,"{2017-04-14 00:00:00, 2017-04-19 00:00:00, 201..."
9,8A,"{2017-05-07 00:00:00, 2017-05-08 00:00:00, 201..."


In [12]:
#create an event table by taking all possible henid and sessionnumber combination that has a severity value
df_event = df[~df['severity'].isnull()][['HenID','sessionNumber','severity','group']].copy()
df_event.drop_duplicates(subset=None, keep='first', inplace=True)
#compute previous info
df_event['HenID_sessionNumber'] = df_event['HenID']+'_'+df_event['sessionNumber'].map(str)
df_event['previous_sessionNumber'] = df_event['sessionNumber'].map(lambda x: x-1)
df_event['previous_HenID_sessionNumber'] = df_event['HenID']+'_'+df_event['previous_sessionNumber'].map(str)
dico_id_sev = dict(zip(df_event['HenID_sessionNumber'].tolist(), df_event['severity'].tolist()))
df_event['previous_severity'] = df_event['previous_HenID_sessionNumber'].map(lambda x: dico_id_sev[x] if x in dico_id_sev \
                                                                             else np.nan)
df_event['sum_previous_severity'] = df_event.apply(lambda x: sum(df_event[(df_event['HenID']==x['HenID']) & \
                                                  (df_event['sessionNumber']<x['sessionNumber'])]['severity'].tolist()), axis=1)
df_event['previous_sessionNumber']= df_event['previous_sessionNumber'].replace(0, np.nan)
df_event['severity-previousSeverity'] = df_event['severity'] - df_event['previous_severity']
print(df_event.shape)
df_event.head(10)

(1169, 10)


,HenID,sessionNumber,severity,group,HenID_sessionNumber,previous_sessionNumber,previous_HenID_sessionNumber,previous_severity,sum_previous_severity,severity-previousSeverity
0,hen_1,1,0.0,A,hen_1_1,NaN,hen_1_0,NaN,0.0,NaN
720,hen_1,2,0.0,A,hen_1_2,1.0,hen_1_1,0.0,0.0,0.0
1520,hen_1,3,0.8,A,hen_1_3,2.0,hen_1_2,0.0,0.0,0.8
2328,hen_1,4,0.3,A,hen_1_4,3.0,hen_1_3,0.8,0.8,-0.5
2828,hen_1,5,0.2,A,hen_1_5,4.0,hen_1_4,0.3,1.1,-0.1
3248,hen_1,6,1.5,A,hen_1_6,5.0,hen_1_5,0.2,1.3,1.3
3476,hen_1,7,1.6,A,hen_1_7,6.0,hen_1_6,1.5,2.8,0.1
3694,hen_1,8,1.5,A,hen_1_8,7.0,hen_1_7,1.6,4.4,-0.1
4412,hen_1,9,1.5,A,hen_1_9,8.0,hen_1_8,1.5,5.9,0.0
4666,hen_1,10,1.8,A,hen_1_10,9.0,hen_1_9,1.5,7.4,0.3


In [13]:
#define interesting event flag
gap = 6
df_event['is_interesting_event'] = df_event.apply(lambda x: (x['severity']-x['previous_severity'])>=gap, axis=1)
df_event['is_interesting_event'] = df_event.apply(lambda x: (x['sum_previous_severity']==0) & (x['severity']>=1), axis=1)
df_event['is_interesting_event'].value_counts()

False    1061
True      108
Name: is_interesting_event, dtype: int64

In [14]:
#define not-interesting event flag
df_event['is_not_interesting_event'] = df_event.apply(lambda x: (x['sum_previous_severity']==0) & (x['severity']==0), axis=1)
df_event['is_not_interesting_event'].value_counts()

False    921
True     248
Name: is_not_interesting_event, dtype: int64

In [15]:
#remove event with no previous sessionnumber or no previous severity(i.e. not ebale to compare) and save
df_event = df_event[(~df_event['previous_sessionNumber'].isnull()) & (~df_event['previous_severity'].isnull())]
#save event table
print(df_event.shape)
df_event.to_csv(os.path.join(path_extracted_data, 'df_event.csv'), sep=';', index=False)
#save interesting events
df_int_event = df_event[df_event['is_interesting_event']].copy()
print(df_int_event.shape)
df_int_event.to_csv(os.path.join(path_extracted_data, 'df_int_event.csv'), sep=';', index=False)
#save not interesting events
df_not_int_event = df_event[df_event['is_not_interesting_event']].copy()
print(df_not_int_event.shape)
df_not_int_event.to_csv(os.path.join(path_extracted_data, 'df_not_int_event.csv'), sep=';', index=False)

(989, 12)
(78, 12)
(146, 12)
